<a href="https://colab.research.google.com/github/vikas-hkk/Lild4o0d8/blob/main/Demo(_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [5]:
# Optimized Skincare Q&A System with RAG
# Clean, modular approach for Google Colab

# Installation (run once)
!pip install -q langchain faiss-cpu sentence-transformers transformers torch

import torch
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import warnings
warnings.filterwarnings('ignore')

class SkincareQA:
    def __init__(self):
        self.embeddings = None
        self.db = None
        self.qa_chain = None
        self._setup_knowledge_base()
        self._setup_model()
        self._create_qa_chain()

    def _setup_knowledge_base(self):
        """Initialize vector database with skincare facts"""
        print(" Setting up knowledge base...")

        skincare_facts = [
            "Hyaluronic acid binds 1000x its weight in water, providing deep hydration by attracting and retaining moisture in skin cells",
            "Retinol accelerates cell turnover and stimulates collagen production, but increases photosensitivity requiring daily SPF protection",
            "Niacinamide (Vitamin B3) reduces inflammation, minimizes pores, and regulates sebum production without causing irritation",
            "Vitamin C brightens skin and provides antioxidant protection, but should be used in morning with sunscreen",
            "Salicylic acid penetrates pores to remove dead skin cells and excess oil, making it effective for acne-prone skin"
        ]

        self.embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
        self.db = FAISS.from_texts(skincare_facts, self.embeddings)
        print(" Knowledge base ready!")

    def _setup_model(self):
        """Initialize the language model with optimized settings"""
        print(" Loading language model...")

        model_name = "microsoft/phi-2"

        # Tokenizer setup
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        if not self.tokenizer.pad_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Model setup
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
            trust_remote_code=True
        )

        # Pipeline with stricter parameters for cleaner output
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=30,  # Reduced for shorter responses
            do_sample=False,
            temperature=0.0,
            pad_token_id=self.tokenizer.eos_token_id,
            truncation=True
        )
        print(" Model loaded!")

    def _create_qa_chain(self):
        """Create the Q&A chain with custom prompt"""
        prompt_template = """Use the following skincare information to answer the question concisely:

Context: {context}

Question: {question}

Answer (one clear sentence):"""

        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )

        llm = HuggingFacePipeline(pipeline=self.pipe)

        self.qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=self.db.as_retriever(search_kwargs={"k": 1}),
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True
        )
        print("Q&A system ready!")

    def ask(self, question):
        """Ask a skincare question and get a clean response"""
        print(f"\n Question: {question}")
        print("-" * 50)

        # Get similar context for transparency
        similar_docs = self.db.similarity_search(question, k=1)
        context = similar_docs[0].page_content
        print(f" Retrieved Context: {context}")

        # Get answer
        result = self.qa_chain({"query": question})
        raw_answer = result['result']

        # Clean the response
        clean_answer = self._clean_response(raw_answer)

        print(f" Answer: {clean_answer}")
        return clean_answer

    def _clean_response(self, text):
        """Extract clean answer from model output"""
        # Find the actual answer after the prompt
        if "Answer (one clear sentence):" in text:
            answer = text.split("Answer (one clear sentence):")[-1]
        elif "Answer:" in text:
            answer = text.split("Answer:")[-1]
        else:
            answer = text

        # Clean up the response - take only the first sentence
        answer = answer.strip()
        if '\n' in answer:
            answer = answer.split('\n')[0]

        # Remove any trailing incomplete text or weird characters
        answer = answer.split('.')[0] + '.' if '.' in answer else answer
        return answer.strip()

# Initialize and test the system
def main():
    print(" Initializing Skincare Q&A System...")
    qa_system = SkincareQA()

    # Test questions - reduced to 2 for clean output
    test_questions = [
        "How does hyaluronic acid work?",
        "Should I use vitamin C in morning or night?"
    ]

    print("\n" + "="*60)
    print(" SKINCARE Q&A DEMO")
    print("="*60)

    for question in test_questions:
        qa_system.ask(question)
        print()

    return qa_system

# Run the system
if __name__ == "__main__":
    skincare_qa = main()

 Initializing Skincare Q&A System...
 Setting up knowledge base...
 Knowledge base ready!
 Loading language model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


 Model loaded!
Q&A system ready!

 SKINCARE Q&A DEMO

 Question: How does hyaluronic acid work?
--------------------------------------------------
 Retrieved Context: Hyaluronic acid binds 1000x its weight in water, providing deep hydration by attracting and retaining moisture in skin cells
 Answer: Hyaluronic acid works by binding 1000x its weight in water, providing deep hydration by attracting and retaining moisture in skin cells.


 Question: Should I use vitamin C in morning or night?
--------------------------------------------------
 Retrieved Context: Vitamin C brightens skin and provides antioxidant protection, but should be used in morning with sunscreen
 Answer: Vitamin C should be used in the morning with sunscreen.



In [4]:
# Streamlined LangChain Cosmetic Advisory with Multi-Agent Setup
# Fixes RetrievalQA validation error and implements proper workflow

# Installation
!pip install -q langchain langgraph faiss-cpu sentence-transformers transformers torch langchain-community

import torch
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import warnings
warnings.filterwarnings('ignore')

class CosmeticAdvisorySystem:
    def __init__(self):
        self.llm = self._setup_llm()
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        self.vector_stores = self._setup_knowledge_base()
        self.agents = self._create_agents()

    def _setup_llm(self):
        """Setup lightweight LLM"""
        print(" Loading model...")

        tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
        tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            "microsoft/phi-2",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            trust_remote_code=True
        )

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=80,
            temperature=0.1,
            pad_token_id=tokenizer.eos_token_id
        )

        return HuggingFacePipeline(pipeline=pipe)

    def _setup_knowledge_base(self):
        """Create vector databases"""
        print(" Building knowledge base...")

        knowledge = {
            "treatments": [
                "Botox: Muscle relaxant injection, reduces wrinkles, lasts 3-6 months, $300-800",
                "Dermal fillers: Hyaluronic acid injections, add volume, last 6-18 months",
                "Chemical peel: Remove skin layers, improve texture, 1-2 weeks recovery",
                "Laser resurfacing: Stimulate collagen, treat scars, 2-4 weeks healing",
                "Rhinoplasty: Nose surgery, 2 weeks recovery, final results in 12 months",
                "Facelift: Tighten facial skin, 2-3 weeks recovery, results last 7-10 years"
            ],
            "safety": [
                "Pre-surgery: Stop smoking 4-6 weeks before, avoid blood thinners",
                "Recovery: Follow wound care, attend follow-ups, report complications",
                "Contraindications: Pregnancy, infections, unrealistic expectations",
                "Warning signs: Excessive bleeding, severe pain, infection signs",
                "Timeline: Initial healing 1-2 weeks, final results 3-12 months"
            ]
        }

        return {
            name: FAISS.from_texts(docs, self.embeddings)
            for name, docs in knowledge.items()
        }

    def _create_agents(self):
        """Create specialized agent chains"""
        print(" Creating agents...")

        # Agent 1: Treatment Specialist
        treatment_prompt = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a cosmetic treatment specialist.

Context: {context}
Question: {question}

Provide treatment details including procedure, duration, and cost.
Response:"""
        )

        # Agent 2: Safety Officer
        safety_prompt = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a medical safety officer.

Context: {context}
Question: {question}

Focus on risks, recovery, and precautions.
Response:"""
        )

        # Agent 3: Consultation Coordinator
        coordinator_prompt = PromptTemplate(
            input_variables=["treatment_info", "safety_info", "question"],
            template="""Create final consultation summary.

Treatment: {treatment_info}
Safety: {safety_info}
Question: {question}

Provide comprehensive recommendation.
Summary:"""
        )

        return {
            "treatment_specialist": LLMChain(llm=self.llm, prompt=treatment_prompt),
            "safety_officer": LLMChain(llm=self.llm, prompt=safety_prompt),
            "coordinator": LLMChain(llm=self.llm, prompt=coordinator_prompt)
        }

    def _get_context(self, question, db_name):
        """Retrieve relevant context from vector DB"""
        docs = self.vector_stores[db_name].similarity_search(question, k=2)
        return " ".join([doc.page_content for doc in docs])

    def _clean_response(self, text):
        """Clean LLM output"""
        for marker in ["Response:", "Summary:", "Answer:"]:
            if marker in text:
                text = text.split(marker)[-1]
        return text.strip().split('\n')[0] if '\n' in text else text.strip()

    def consult(self, question):
        """Multi-agent consultation workflow"""
        print(f"\n CONSULTATION: {question}")
        print("-" * 50)

        # Step 1: Treatment Specialist Analysis
        print(" Treatment Specialist analyzing...")
        treatment_context = self._get_context(question, "treatments")
        treatment_response = self.agents["treatment_specialist"].run({
            "context": treatment_context,
            "question": question
        })
        treatment_info = self._clean_response(treatment_response)

        # Step 2: Safety Officer Assessment
        print(" Safety Officer reviewing...")
        safety_context = self._get_context(question, "safety")
        safety_response = self.agents["safety_officer"].run({
            "context": safety_context,
            "question": question
        })
        safety_info = self._clean_response(safety_response)

        # Step 3: Consultation Coordinator Summary
        print(" Coordinator creating summary...")
        final_response = self.agents["coordinator"].run({
            "treatment_info": treatment_info,
            "safety_info": safety_info,
            "question": question
        })
        final_summary = self._clean_response(final_response)

        # Display Results
        print(f"\n TREATMENT ANALYSIS:")
        print(f"   {treatment_info}")

        print(f"\n  SAFETY ASSESSMENT:")
        print(f"   {safety_info}")

        print(f"\n FINAL RECOMMENDATION:")
        print(f"   {final_summary}")

        return {
            "treatment": treatment_info,
            "safety": safety_info,
            "recommendation": final_summary
        }

# Quick Demo
def demo():
    system = CosmeticAdvisorySystem()

    questions = [
        "How much does Botox cost and how long does it last?",
        "What are the risks of rhinoplasty surgery?",
        "Is a facelift safe for someone over 60?"
    ]

    for q in questions:
        result = system.consult(q)
        print("\n" + "="*60 + "\n")

if __name__ == "__main__":
    demo()

 Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


 Building knowledge base...
 Creating agents...

 CONSULTATION: How much does Botox cost and how long does it last?
--------------------------------------------------
 Treatment Specialist analyzing...
 Safety Officer reviewing...
 Coordinator creating summary...

 TREATMENT ANALYSIS:
   Botox is a muscle relaxant injection that reduces wrinkles. It lasts for 3-6 months and costs between $300 and $800.

  SAFETY ASSESSMENT:
   Botox is a neurotoxin that paralyzes muscles and reduces wrinkles. It is injected into the skin with a fine needle. The cost of Botox varies depending on the area, the number of units, and the doctor. It can range from $300 to $1000 per session. The effects of Botox last for about 3-4 months, but they can vary depending on the individual and the

 FINAL RECOMMENDATION:
   Botox is a muscle relaxant injection that reduces wrinkles. It costs between $300 and $800 per session, depending on the area, the number of units, and the doctor. The effects of Botox last for 